In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/Project_2_DPL

BiLSTM.py   disvoice	  kaldi_io		    mfcc.py	prosody.py   test.py
CAT.py	    HuBERT.py	  kaldi_io-0.9.8.dist-info  phonet	__pycache__  utils.py
dataset.py  hump_cat.pth  main_model.py		    predict.py	pysptk


In [3]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=0b06280b2b0d8afcbe4331e9bbc3fcabefa065d8716a4a6fed561d9f233ed720
  Stored in directory: /root/.cache/pip/wheels/37/01/19/e6c69a32684ab7b2e3ea4985a571d810cf055c72600e7f9f17
Successfully built python_speech_features


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Project_2_DPL')


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataset import RAVDESSDataset
from torch.utils.data import DataLoader
from main_model import HuMP_CAT
from utils import collate_fn

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
#!pip install phonet
#!mkdir -p /content/drive/MyDrive/Project_2_DPL
#!cp -r /usr/local/lib/python3.*/dist-packages/phonet /content/drive/MyDrive/Project_2_DPL

In [ ]:
#!pip install pysptk
#!mkdir -p /content/drive/MyDrive/Project_2_DPL
#!cp -r /usr/local/lib/python3.*/dist-packages/pysptk /content/drive/MyDrive/Project_2_DPL

\

In [6]:
if torch.cuda.is_available():
    print("🔵 Training on GPU:", torch.cuda.get_device_name(0))
else:
    print("🔴 Training on CPU")


🔵 Training on GPU: NVIDIA L4


In [19]:
folder_path = r"/content/drive/MyDrive/RAVDESS_dataset/ravdess/train"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = RAVDESSDataset(folder_path, device)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_fn)

num_classes = 8  # Số nhãn trong dataset


# 🔹 Train loop

KeyboardInterrupt: 

In [58]:
from collections import Counter

counts = Counter(train_data.labels)
print(counts)  # Trả về một dictionary {giá trị unique: số lần xuất hiện}

Counter({'calm': 157, 'disgust': 157, 'happy': 155, 'angry': 155, 'surprised': 154, 'fearful': 149, 'sad': 148, 'neutral': 77})


In [68]:
model = HuMP_CAT(embed_dim=256, num_heads=4, embed_dim_x1=256, embed_dim_x2=256, num_classes=num_classes).to(device)

# 🔹 Loss function và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)

In [73]:
num_epochs = 22
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for mfcc, prosody, hubert, labels in train_loader:
        hubert, mfcc, prosody, labels = hubert.to(device), mfcc.to(device), prosody.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(hubert, mfcc, prosody, device)

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        labels = labels.argmax(dim=1)  # Chuyển từ one-hot về label index
        correct += (logits.argmax(dim=1) == labels).sum().item()

        total += labels.size(0)

    acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Acc: {acc:.4f}")

Epoch [1/2], Loss: 24.4214, Acc: 0.9106
Epoch [2/2], Loss: 20.0730, Acc: 0.9340


In [74]:
model_path = r"/content/drive/MyDrive/Project_2_DPL/hump_cat.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ Mô hình đã được lưu vào: {model_path}")

✅ Mô hình đã được lưu vào: /content/drive/MyDrive/Project_2_DPL/hump_cat.pth
